# Percobaan 4 (GloVe NN(LSTM))

---



In [1]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 1.9 MB/s eta 0:00:00


In [3]:
!pip install tensorflow
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 55.0 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df = pd.read_csv('/content/purbaya_purbaya_labeled.csv')

In [6]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()
factory_stop = StopWordRemoverFactory()
stop_words = factory_stop.get_stop_words()

# 2. Fungsi Preprocessing
def preprocess_teks(teks):
    if not isinstance(teks, str): # Cek agar tidak error jika data kosong
        return ""

    teks = teks.lower()
    teks = re.sub(r'[^a-zA-Z0-9\s]', '', teks)
    tokens = teks.split()

    # Filtering & Stemming
    final_tokens = []
    for token in tokens:
        # Sekarang 'stop_words' adalah list, jadi 'not in' akan bekerja
        if token not in stop_words:
            stemmed_token = stemmer.stem(token)
            final_tokens.append(stemmed_token)

    return ' '.join(final_tokens)

df['clean_teks'] = df['normalized_text'].apply(preprocess_teks)

In [9]:
le = LabelEncoder()
y = le.fit_transform(df['label_manual'])
print("Mapping Label:", dict(zip(le.classes_, le.transform(le.classes_))))

Mapping Label: {'negatif': np.int64(0), 'netral': np.int64(1), 'positif': np.int64(2)}


In [24]:
embeddings_index = {}
f = open('/content/glove.6B.100d.txt', encoding='utf8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print(f"Selesai. Ditemukan {len(embeddings_index)} vektor kata.")

Selesai. Ditemukan 400000 vektor kata.


In [25]:
max_features = 5000  # Maksimal kosakata unik
max_len = 50         # Panjang kalimat seragam

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['clean_teks'])
X = tokenizer.texts_to_sequences(df['clean_teks'])
X = pad_sequences(X, maxlen=max_len)

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_dim = 100

# Mapping kata di dataset ke vektor GloVe
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [30]:
model = Sequential()
# Layer Embedding menggunakan bobot GloVe (trainable=False agar bobot awal tidak rusak)
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
print("\nMulai Training Model...")
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)


Mulai Training Model...
Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 376ms/step - accuracy: 0.4597 - loss: 1.0766 - val_accuracy: 0.7733 - val_loss: 0.9183
Epoch 2/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 121ms/step - accuracy: 0.7735 - loss: 0.8777 - val_accuracy: 0.7733 - val_loss: 0.7804
Epoch 3/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.7194 - loss: 0.8227 - val_accuracy: 0.7733 - val_loss: 0.7313
Epoch 4/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step - accuracy: 0.7791 - loss: 0.7002 - val_accuracy: 0.7867 - val_loss: 0.7103
Epoch 5/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 108ms/step - accuracy: 0.7512 - loss: 0.6774 - val_accuracy: 0.7867 - val_loss: 0.7143
Epoch 6/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 107ms/step - accuracy: 0.7433 - loss: 0.7091 - val_accuracy: 0.7867 - val_loss: 0.6927
Epoch 7/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 101ms/step - accuracy: 0.7906 - loss: 0.5898 - val_accuracy: 0.7867 - val_loss: 0.6618
Epoch 8/100
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 110ms/step - accuracy: 0.7668 - loss: 0.5925 - val_accuracy: 0.7867 - val_loss: 0.6

In [31]:
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step

3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 221ms/step

Laporan Klasifikasi:
              precision    recall  f1-score   support

     negatif       0.50      0.17      0.25         6
      netral       0.81      0.85      0.83        59
     positif       0.18      0.20      0.19        10

    accuracy                           0.71        75
   macro avg       0.50      0.40      0.42        75
weighted avg       0.70      0.71      0.70        75

